## Introduction

There are many ways data can end up with missing values. For example,

    A 2 bedroom house won't include a value for the size of a third bedroom.
    A survey respondent may choose not to share his income.

Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. 

So you'll need to choose one of the strategies below.

## Three Approaches¶
###### 1) A Simple Option: Drop Columns with Missing Values

The simplest option is to drop columns with missing values.

###### 2) A Better Option: Imputation

Imputation fills in the missing values with some number. 

For instance, we can fill in the mean value along each column.

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get 

from dropping the column entirely.

#### 3) An Extension To Imputation

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('/root/Downloads//melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)



In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
def score_predict(train_x,train_y,val_x,val_y):
    melb_modal=RandomForestRegressor(n_estimators=10,random_state=1)
    melb_modal.fit(train_x,train_y)
    pred=melb_modal.predict(val_x)
    return mean_absolute_error(val_y,pred)

In [13]:
col_with_missing_value=[col for col in X_train.columns if X_train[col].isnull().any() ]

In [14]:
col_with_missing_value

['Car', 'BuildingArea', 'YearBuilt']

In [18]:
x_train_witout_missing_value=X_train.drop(col_with_missing_value,axis=1)
x_value_without_missing_value=X_valid.drop(col_with_missing_value,axis=1)

In [20]:
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_predict(x_train_witout_missing_value,y_train,x_value_without_missing_value,y_valid))

MAE from Approach 1 (Drop columns with missing values):
183207.24410372396


#### score from Approach 2 (Imputation)

Next, we use SimpleImputer to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). 

While statisticians have experimented with more complex ways to determine imputed values (such as regression imputation, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.


In [26]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer()
x_train_imputer=pd.DataFrame(imputer.fit_transform(X_train))
x_valid_imputer=pd.DataFrame(imputer.transform(X_valid))
x_train_imputer.columns=X_train.columns
x_valid_imputer.columns=X_valid.columns
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_predict(x_train_imputer,y_train,x_valid_imputer,y_valid))


MAE from Approach 1 (Drop columns with missing values):
178224.83037029245


#### Score from Approach 3 (An Extension to Imputation)

Next, we impute the missing values, while also keeping track of which values were imputed.

In [32]:
X_train_plus=X_train.copy()
X_valid_plus=X_valid.copy()
for col in col_with_missing_value:
    X_train_plus[col+"_Missing"]=X_train_plus[col].isnull()
    X_train_plus[col+"_Missing"]=X_valid_plus[col].isnull()
    

In [33]:
imputer=SimpleImputer()
imp_x_train=pd.DataFrame(imputer.fit_transform(X_train_plus))

In [36]:
X_train_plus.columns

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount', 'Car_Missing', 'BuildingArea_Missing',
       'YearBuilt_Missing'],
      dtype='object')

In [39]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in col_with_missing_value:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_predict(imputed_X_train_plus,y_train, imputed_X_valid_plus ,y_valid))

MAE from Approach 3 (An Extension to Imputation):
178950.41952275755


In [40]:
print(X_train.shape)

(10864, 12)


In [42]:
missing_values_count=X_train.isnull().sum()

In [48]:
X_train.isnull().mean()

Rooms            0.000000
Distance         0.000000
Postcode         0.000000
Bedroom2         0.000000
Bathroom         0.000000
Car              0.004510
Landsize         0.000000
BuildingArea     0.474595
YearBuilt        0.396447
Lattitude        0.000000
Longtitude       0.000000
Propertycount    0.000000
dtype: float64